In [114]:
%load_ext autoreload
%autoreload 2

import re, os, sys
#sys.path.insert(0, os.path.realpath('/content/jupyter/mta-accessibility/notebooks/routing'))
sys.path.insert(0, os.getcwd())
from gcs_utils import gcs_util
import GTFS_Utils as gu

import os.path
from os import listdir
from os.path import isfile, join
from google.transit import gtfs_realtime_pb2 as gtfs_rt
from protobuf_to_dict import protobuf_to_dict
import datetime
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize
from itertools import groupby
import requests
from datetime import datetime
from typing import List, Dict, Tuple

pd.set_option('max_colwidth', 200)
gcs = gcs_util()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [113]:
def parse_blob_name(name: str):
    """Returns msg_type, date, time"""
    pat = re.compile("realtime\/(.*)_([0-9]*)-([0-9]*)")
    return pat.match(name).groups()


blobs_by_date = {}
for blob in gcs.list_blobs('realtime'):
    
    msg_type, date, time = parse_blob_name(blob.name)
    blobs_by_date.setdefault(date, []).append(blob.name)    

<Blob: nycelevator.dataclinic.research.alphastudio.com, realtime/hourly/vehicle_updates_20200419_11, 1588890446206383>

In [101]:
import logging

class Plog:    
    def __init__(self):        
        log = logging.getLogger('gtfs_parser')
        log.setLevel(logging.DEBUG)

        filename = "realtime{}.log".format(datetime.now().strftime("%Y%m%d-%I%M%S"))
        # create file handler which logs even debug messages
        fh = logging.FileHandler(filename)
        fh.setLevel(logging.DEBUG)

        # create console handler with a higher log level
        ch = logging.StreamHandler()
        ch.setLevel(logging.ERROR)

        # create formatter and add it to the handlers
        formatter = logging.Formatter('%(asctime)s-%(levelname)s: %(message)s')
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)

        if log.hasHandlers():
            log.handlers.clear()
        # add the handlers to the logger
        log.addHandler(fh)
        log.addHandler(ch)

        self.log_instance = log
                    
    def error(self, *args, **kwargs):
        self.log_instance.error(*args, **kwargs)
    def info(self, *args, **kwargs):
        self.log_instance.info(*args, **kwargs)
    def warn(self, *args, **kwargs):
        self.log_instance.warning(*args, **kwargs)
        
log = Plog()

In [102]:
# Stops data
stop_df = gcs.read_dataframe('google_transit/stops.pkl')[['stop_id', 'stop_name']]    

def parse_vehicle_updates(realtime_data: List[Dict]) -> pd.DataFrame:
    df = json_normalize(realtime_data)    
    df = df[df['vehicle.trip.trip_id'].notnull()]        
    df = df.drop(list(df.filter(regex='trip_update')), axis=1)
    df.columns = df.columns.str.replace("vehicle\.", '')
    df.columns = df.columns.str.replace("trip\.", '')
    df = df.merge(stop_df, on='stop_id')
    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s')
    return df.drop_duplicates()
    
def parse_trip_updates(realtime_data: List[Dict]) -> pd.DataFrame:
    """
    Returns trip_updates
    """    
    df = json_normalize(realtime_data)    
    
    # Parse trip updates
    df = df[~df['trip_update.stop_time_update'].isnull()]
    idx = df.index.repeat(df['trip_update.stop_time_update'].str.len())
    df1 = pd.DataFrame({'trip_update.stop_time_update': np.concatenate(df['trip_update.stop_time_update'].values)})
    df1.index = idx
    df = df1.join(df.drop('trip_update.stop_time_update', axis=1), how='left')
    df = df.reset_index(drop=True)
    d2 = json_normalize(df['trip_update.stop_time_update'])
    df = pd.concat([d2, df.drop('trip_update.stop_time_update', axis=1)], axis=1)
    df = df.drop(list(df.filter(regex='vehicle')), axis=1) 
    df = df.merge(stop_df, on='stop_id')
    
    df.columns = df.columns.str.replace("trip_update\.trip\.", '')
    df.columns = df.columns.str.replace("\.", '_')
    df['arrival_time'] = pd.to_datetime(df.arrival_time, unit='s')
    df['departure_time'] = pd.to_datetime(df.departure_time, unit='s')
        
    return df.drop_duplicates()

In [91]:

#stop_df[stop_df.stop_id.str.contains('L')]

In [ ]:
api_key = "06d19c93248b50269add7ffeeace3796"
import time
# Train lines to feed_id
feed_id_dict = {
    "123456S": "1",
    "acehs": "26",
    "nqrw": "16",
    "bdfm": "21",
    #"l": "2",
    "g": "31",
    "jz": "36",
    "7": "51",    
}

while True:
    tdf = pd.DataFrame()
    vdf = pd.DataFrame()
    for train_names, feed_id in feed_id_dict.items():        
        url = "http://datamine.mta.info/mta_esi.php?key={}&feed_id={}".format(api_key, feed_id)
        r = requests.get(url, allow_redirects=True)
        try:
            realtime = gu.gtfs_str_to_dict(r.content)    
            vdf = vdf.append(parse_vehicle_updates(realtime), sort=True)
            tdf = tdf.append(parse_trip_updates(realtime), sort=True)
        except Exception as ex:
            log.error(ex)
            continue

    time_str = datetime.now().strftime("%Y%m%d-%H%M%S")
    vehicle_path = f"realtime/raw_df/vehicle_updates_{time_str}"
    trip_path = f"realtime/raw_df/trip_updates_{time_str}"

    gcs.write_dataframe(vdf, vehicle_path)
    gcs.write_dataframe(tdf, trip_path)

    log.info(f"Wrote {len(vdf.index)} rows to {vehicle_path}")
    log.info(f"Wrote {len(tdf.index)} rows to {trip_path}\n")

    time.sleep(60)

2020-05-13 13:47:17,573-ERROR: Error parsing raw string
2020-05-13 14:21:31,907-ERROR: Error parsing raw string
2020-05-13 15:53:39,797-ERROR: Error parsing raw string
2020-05-13 17:01:09,389-ERROR: Error parsing raw string


,id,current_status,current_stop_sequence,timestamp,route_id,start_date,trip_id
1,2,1.0,20.0,2020-04-17 17:21:09,L,20200417,077100_L..N
3,4,1.0,20.0,2020-04-17 17:20:39,L,20200417,077200_L..S
5,6,1.0,15.0,2020-04-17 17:21:09,L,20200417,078100_L..N
7,8,1.0,13.0,2020-04-17 17:20:24,L,20200417,078200_L..S
9,10,1.0,7.0,2020-04-17 17:21:09,L,20200417,079100_L..N
